In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle


2023-11-13 11:34:46.780719: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_path = '/Users/PremGanesh/Developer/AI/CyVidia/Trail 8/trained_model_rbi_jll_nist'
model = load_model(model_path)

with open('/Users/PremGanesh/Developer/AI/CyVidia/Trail 8/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('/Users/PremGanesh/Developer/AI/CyVidia/Trail 8/area_encoder.pickle', 'rb') as handle:
    area_encoder = pickle.load(handle)

with open('/Users/PremGanesh/Developer/AI/CyVidia/Trail 8/bucket_encoder.pickle', 'rb') as handle:
    bucket_encoder = pickle.load(handle)


In [3]:
def clean_text(text):
    if isinstance(text, float) and np.isnan(text):
        return ""
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalnum()]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    clean_text = ' '.join(words)
    return clean_text


In [4]:
excel_file_path = '/Users/PremGanesh/Developer/AI/CyVidia/Requirement_extraction/claro_requirement_data.xlsx'
df = pd.read_excel(excel_file_path)


In [5]:
df['Cleaned_Sentence'] = df['Sentence'].apply(clean_text)


In [6]:
tokenized_sentences = tokenizer.texts_to_sequences(df['Cleaned_Sentence'])
padded_sentences = pad_sequences(tokenized_sentences, maxlen=100)


In [1]:
sentence_predictions = model.predict(padded_sentences)
# check the length of the predictions
print(len(sentence_predictions)) 
# check the lenght of the dataframe
print(len(df))
# Assuming your model outputs probabilities for different areas, you can threshold them to get binary predictions
threshold = 0.5
binary_sentence_predictions = [(area > threshold).astype(int) for area in sentence_predictions]

# Decode the predictions using the label encoders
sentence_labels = [area_encoder.classes_[prediction[0]] for prediction in binary_sentence_predictions]

# Make sure the length of sentence_labels matches the length of the DataFrame
if len(sentence_labels) == len(df):
    # Add the predicted labels to the DataFrame
    df['Related_to_Cybersecurity'] = sentence_labels
else:
    print("Error: Length of predictions does not match length of DataFrame.")


NameError: name 'model' is not defined